In [11]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
import scipy.stats as st
import datetime

from scipy.stats import linregress
from pprint import pprint

from api_keys import weather_api_key
from api_keys import geoapify_key

from citipy import citipy


# In[2]:


#import csv file
resort_city_data_df = pd.read_csv("resort_city_crime_weather_df.csv", index_col="City_ID")

resort_city_data_df


# In[3]:


#cleaning up column names
resort_city_data_df.rename(columns={"Country_x" : "Country"}, inplace=True)
resort_city_data_df.drop(columns="Country_y", inplace=True)
resort_city_data_df


# In[4]:


#setting climate parameters for ideal locations
resort_city_ideal_df = resort_city_data_df.loc[(resort_city_data_df["Max Temp"] < 32.2)
                                 & (resort_city_data_df["Max Temp"] > 24)
                                & (resort_city_data_df["Humidity"] < 70) 
                                & (resort_city_data_df["Humidity"] > 20)
                                & (resort_city_data_df["Wind Speed"] < 8)
                                & (resort_city_data_df["Cloudiness"] == 0)]

resort_city_ideal_df


# In[5]:


#see how many cities are in the dataframe
resort_city_ideal_df.count()


# In[17]:


resort_city_ideal_hotel_df = resort_city_ideal_df.copy()

resort_city_ideal_hotel_df["Number of Hotels"] = ""
resort_city_ideal_hotel_df["Number of Resturants"] = ""
resort_city_ideal_hotel_df["Bodies of Water"] = ""
resort_city_ideal_hotel_df["Tourist Attractions"] = ""
resort_city_ideal_hotel_df["Natural Places"] = ""
resort_city_ideal_hotel_df["Hospital"] = ""
resort_city_ideal_hotel_df["Entertainment"] = ""
resort_city_ideal_hotel_df["Rental Car"] = ""
resort_city_ideal_hotel_df["Airport"] = ""

resort_city_ideal_hotel_df.head(200)



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Rank,...,Safety Index,Number of Hotels,Number of Resturants,Bodies of Water,Tourist Attractions,Natural Places,Hospital,Entertainment,Rental Car,Airport
City_ID,,,,,,,,,,,,,,,,,,,,,
41,newman,37.3138,-121.0208,27.81,45,0,0.89,US,2023-04-28 10:40:44,NaN,...,NaN,,,,,,,,,
48,laguna,38.4210,-121.4238,29.49,50,0,1.54,US,2023-04-28 10:40:45,NaN,...,NaN,,,,,,,,,
60,los altos,37.3852,-122.1141,28.10,64,0,3.60,US,2023-04-28 10:38:03,NaN,...,NaN,,,,,,,,,
68,capayan,-28.7610,-66.0502,31.72,31,0,1.41,AR,2023-04-28 10:40:50,NaN,...,NaN,,,,,,,,,
74,cabo san lucas,22.8909,-109.9124,30.61,36,0,3.60,MX,2023-04-28 10:40:52,NaN,...,NaN,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3779,artigas,-30.4000,-56.4667,27.63,30,0,4.76,UY,2023-04-28 11:01:26,NaN,...,NaN,,,,,,,,,
3798,santa rosa misiones,-26.8667,-56.8500,26.24,41,0,2.27,PY,2023-04-28 11:02:25,NaN,...,NaN,,,,,,,,,
3803,villa constitucion,-33.2278,-60.3297,28.32,28,0,5.74,AR,2023-04-28 11:02:27,NaN,...,NaN,,,,,,,,,


In [12]:

def city_categories(categories, column, df):
    
    for index, row in df.iterrows():
        # get latitude, longitude from the DataFrame
        latitude = row["Lat"]
        longitude = row["Lng"]

        # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary

        # Set base URL
       
        radius = 16000

        # Set the parameters for the type of search
        filters = f"circle:{longitude},{latitude},{radius}"
        bias = f"proximity:{longitude},{latitude}"
        limit = 200

        # set up a parameters dictionary
        params = {
            "categories":categories,
            "limit":limit,
            "filter":filters,
            "bias":bias,
            "apiKey":geoapify_key    
        }

        # Set base URL
        base_url = "https://api.geoapify.com/v2/places"

        # run a request using our params dictionary
        response = requests.get(base_url, params=params)
        response_json = response.json()

        #pprint(response_json)


        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            df.loc[index, column] = len(response_json["features"])
        except (KeyError, IndexError):
            df.loc[index, column] = 0   
        
    return df 


In [ ]:


# In[ ]:


categories = "airport.international"
column = "Airport"
resort_city_ideal_hotel_df = city_categories(categories, column, resort_city_ideal_hotel_df)


# In[34]:


cities_airports = resort_city_ideal_hotel_df[(resort_city_ideal_hotel_df[["Airport"]] != 0).all(axis=1)]


# In[35]:


cities_airports


# In[39]:


categories = "accommodation, beach.beach_resort"
column = "Number of Hotels"
ideal_cities_all_df = city_categories(categories, column, cities_airports)


# In[40]:


categories = "natural.water.sea,natural.water.reef,natural.water"
column = "Bodies of Water"
ideal_cities_all_df = city_categories(categories, column, cities_airports)


# In[41]:


categories = "tourism"
column = "Tourist Attractions"
ideal_cities_all_df = city_categories(categories, column, cities_airports)


# In[42]:


categories = "catering.restaurant"
column = "Number of Resturants"
ideal_cities_all_df = city_categories(categories, column, cities_airports)


# In[43]:


categories = "natural"
column = "Natural Places"
ideal_cities_all_df = city_categories(categories, column, cities_airports)


# In[44]:


categories = "healthcare.hospital"
column = "Hospital"
ideal_cities_all_df = city_categories(categories, column, cities_airports)


# In[45]:


categories = "rental.car"
column = "Rental Car"
ideal_cities_all_df = city_categories(categories, column, cities_airports)


# In[46]:


categories = "entertainment"
column = "Entertainment"
ideal_cities_all_df = city_categories(categories, column, cities_airports)


# In[47]:



ideal_cities_all_df


# In[101]:





In [ ]:

#sorting for  lowest hotels
#cities_sorted_by_hotels = ideal_cities_all_df.sort_values["Number of Hotels"] <= 40], ascending=True)

#cities_sorted_by_hotels

# sort ideal_cities_all_df by "Number of Hotels" in ascending order
sorted_cities = ideal_cities_all_df.sort_values("Number of Hotels")

# filter the results to show only cities with 40 or fewer hotels
filtered_cities = sorted_cities[(sorted_cities["Number of Hotels"] <= 25) & (sorted_cities["Number of Hotels"] >= 4)]

# display the filtered results
filtered_cities

# In[103]:

In [ ]:
filtered_cities.to_csv("new_cities_all_info.csv", index_label="City_ID")